In [12]:
%matplotlib inline
import numpy as np
import pandas as pd

from scipy.optimize import minimize

import cython
%load_ext Cython


%load_ext line_profiler
import line_profiler

from Cython.Compiler.Options import directive_defaults

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [13]:
import KF1
import KF2
import KF_Paper
import KF_PaperUni
import KF3
import CKF3
import CKF4
import CKF5
import CKF6
import CKF7
import CKF8
import CKF9
import CKF10
import CKF11
import CKF12
import CKF13
import CKF14
import CKF15
import CKF16
import CKF17
import CKF18
import CKF19
import CKF20
import CKF21
import CKF22
import CKF23
import CKF24
import CKF25

# Kalman filter

Based on Durbin Koopman


For generic state-space systems like (Hamilton's notation):



$$\underset{(p \times 1)}{y_t} = \underset{(p \times m)}{Z_t} \underset{(m \times 1)}{\alpha_t} + \underset{(p \times 1)}{\epsilon_t}, \qquad \epsilon_t \sim N(0,\underset{(p \times p)}{H_t}) $$

$$\underset{(m \times 1)}{\alpha_{t+1}} = \underset{(m \times m)}{T_t} 
\underset{(m \times 1)}{\alpha_{t}} + \underset{(m \times r)}{R_t} \underset{(r \times r)}{\eta_t}, \qquad \eta_t \sim N(0,\underset{(r \times \ r)}{Q_t})$$


$$\alpha_1 \sim N(a_1,\underset{(m \times m)}{P_1})$$

Where

* $p$ is the number of observed variables
* $m$ is the number of latent states
* $r$ is the number of disturbances (?)


# Dummy data for the KF with non time-varying parameters

## Local Level model

$$ y_t = \mu_t + \epsilon_t, \qquad \epsilon \sim NID(0, \sigma^2_ \epsilon) $$
$$ \mu_{t+1} = \mu_t + \eta_t, \qquad \eta_t \sim NID(0,\sigma^2_ \eta) $$

# Dummy data

In [81]:
# PARAMETERS
m = 2
p = 4


Z = [[0.3,0.7],[0.1,0],[0.5,0.5],[0,0.3]]
# Z = [[1.0,0.0],[0.0,1.0]]


Z = pd.DataFrame(Z)

H = pd.DataFrame(np.diag([0.1,0.2,0.3,0.4]))
# H = pd.DataFrame(np.diag([0.1,0.2,]))


T = pd.DataFrame(np.identity(2))
R = pd.DataFrame(np.identity(2))

Q = pd.DataFrame(np.diag([0.2,0.4]))

# GENERATE DATA

n = 3000 #sample size
# n = 100000
mu = np.empty((n,m))
y = np.empty((n,p))
t = 0
mu[t,:] = np.zeros(m)
y[t,:] = np.zeros(p)

for t in range(1,n):
    mu[t,:] = mu[t-1,:] + np.multiply(np.random.randn(m), np.diag(Q)**0.5)
    y[t,:] = Z.dot(mu[t,:]) + np.multiply(np.random.randn(p), np.diag(H)**0.5)

y = pd.DataFrame(y)    
    
nny = y.copy()
probNan= 0.50
for i in nny.index:
    ran = np.random.uniform(size=nny.iloc[i].shape)
    nny.iloc[i][ran<probNan] = np.nan
    
    
# GENERATE WHOLE NULL

wny = y.copy()
probNan= 0.50
for i in wny.index:
    ran = np.random.uniform(size=(1))
    if ran < probNan:
        wny.iloc[i] = np.nan
    

In [82]:
nny.head(2)

,0,1,2,3
0,NaN,0.0,0.000000,0.0
1,0.155557,NaN,0.348075,NaN


In [83]:
y = np.array(y)
nny = np.array(nny)
wny = np.array(wny)
Z = np.array(Z)
H = np.array(H)
T = np.array(T)
R = np.array(R)
Q = np.array(Q)

#a1 = (m x 1)
a1 = np.zeros(m)
#P1 = (m x m)
P1 = np.diag(np.ones(m)*1.0)

# Likelihood Test



In [84]:
Z[0,0]

0.29999999999999999

In [85]:
Z[1,0]

0.10000000000000001

In [86]:
H[1,1]

0.20000000000000001

In [87]:
def wrapper(z):
    Z2 = Z.copy()
    Z2[0,0] = z[0]
    Z2[1,0] = z[1]
#     Z2[0,0] = z
    H2 = H.copy()
    H2[1,1] = z[2]
#     H2[1,1] = z

#     return -CKF25.KalmanFilter(nny,Z2,H2,T,Q,a1,P1, R, True)
    return -CKF25.KalmanFilter(y,Z2,H2,T,Q,a1,P1, R, True)

In [108]:
x0 = [0.10,0.10,0.10,]
# x0=0
bounds = ((None,None),(None,None),(0,None),)
options = {'maxiter':2000}
# options = {}

In [109]:
minimize(wrapper,x0,method='Nelder-Mead', options=options)
# Doesnt accept bounds
# minimize(wrapper,x0,method='Nelder-Mead',bounds=bounds, options=options) 

 final_simplex: (array([[ 0.29640708,  0.10076162,  0.19984788],
       [ 0.29639742,  0.100757  ,  0.19982684],
       [ 0.29641188,  0.10075669,  0.19994661],
       [ 0.29639149,  0.10076647,  0.19987722]]), array([ 10755.27348567,  10755.27350344,  10755.27352075,  10755.2735343 ]))
           fun: 10755.27348567455
       message: 'Optimization terminated successfully.'
          nfev: 155
           nit: 90
        status: 0
       success: True
             x: array([ 0.29640708,  0.10076162,  0.19984788])

In [110]:
minimize(wrapper,x0,method='Powell', options=options)
# Doesnt accept bounds
# minimize(wrapper,x0,method='Powell',bounds=bounds, options=options)

   direc: array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])
     fun: array(8897.13780777655)
 message: 'Optimization terminated successfully.'
    nfev: 234
     nit: 3
  status: 0
 success: True
       x: array([  2.96451306e-01,   1.56458904e+04,  -7.87874751e+10])

In [111]:
# minimize(wrapper,x0,method='CG', options=options)
minimize(wrapper,x0,method='CG',bounds=bounds, options=options)


C:\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:394: RuntimeWarning: Method CG cannot handle constraints nor bounds.
  RuntimeWarning)


     fun: 10755.27342975272
     jac: array([ 0.00952148, -0.00317383,  0.00061035])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 556
     nit: 28
    njev: 109
  status: 2
 success: False
       x: array([ 0.29640882,  0.10075324,  0.19988048])

In [112]:
# minimize(wrapper,x0,method='BFGS', options=options)
minimize(wrapper,x0,method='BFGS',bounds=bounds, options=options)

C:\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:394: RuntimeWarning: Method BFGS cannot handle constraints nor bounds.
  RuntimeWarning)


      fun: 10755.273429752657
 hess_inv: array([[  3.93399746e-06,  -8.54206374e-07,  -2.97273225e-06],
       [ -8.54206374e-07,   8.88338381e-07,   2.92977262e-07],
       [ -2.97273225e-06,   2.92977262e-07,   2.88564150e-06]])
      jac: array([ 0.00256348, -0.00134277,  0.00170898])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 326
      nit: 13
     njev: 63
   status: 2
  success: False
        x: array([ 0.29640879,  0.10075324,  0.19988046])

In [113]:
# minimize(wrapper,x0,method='L-BFGS-B', options=options)
minimize(wrapper,x0,method='L-BFGS-B',bounds=bounds, options=options)

      fun: 10755.27342981293
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0691216 , -0.22992026,  0.04947651])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 60
      nit: 10
   status: 0
  success: True
        x: array([ 0.29640868,  0.10075306,  0.19988176])

In [114]:
# minimize(wrapper,x0,method='TNC', options=options)
minimize(wrapper,x0,method='TNC',bounds=bounds, options=options)

     fun: 10755.273430231002
     jac: array([ 0.2066372 , -0.36980055, -0.14897523])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 93
     nit: 20
  status: 1
 success: True
       x: array([ 0.29640974,  0.10075277,  0.19987647])

In [115]:
minimize(wrapper,x0,method='SLSQP', options=options)
# minimize(wrapper,x0,method='SLSQP',bounds=bounds, options=options)

     fun: 8897.606231914222
     jac: array([ 0.48583984, -0.00488281, -0.00109863])
 message: 'Optimization terminated successfully.'
    nfev: 654
     nit: 105
    njev: 105
  status: 0
 success: True
       x: array([  2.96443016e-01,   1.06804565e+00,  -3.40920437e+02])

# Testing the KFs

In [ ]:
ret = CKF25.KalmanFilter(nny,Z,H,T,Q,a1,P1, R, False)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF24.KalmanFilter(nny,Z,H,T,Q,a1,P1, R, False)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF23.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF22.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF21.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF20.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF19.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF18.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF17.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF16.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF15.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF14.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF13.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF12.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF11.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF10.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF9.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF8.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF7.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF6.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF5.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF4.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = CKF3.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = KF3.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = KF2.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = KF1.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
ret = KF_PaperUni.KalmanFilter(
    y = y,
#     y = wny,
#     y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
    nStates = a1.shape[0],
    export=True
)

vy = pd.DataFrame(y)

i = 0

ret = ret['yhat']

pd.concat([vy[i], ret[i]], axis=1).plot()

# Performance

In [ ]:
%%timeit

ret = KF_Paper.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
    nStates = a1.shape[0]
)

In [ ]:
%%timeit

ret = KF_PaperUni.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
    nStates = a1.shape[0],
    export=True
)

In [ ]:
%%timeit

ret = KF1.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
)

In [ ]:
%%timeit

ret = KF2.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
)

In [ ]:
%%timeit

ret = KF3.KalmanFilter(
#     y = y,
#     y = wny,
    y = nny,
    Z = Z,
    H = H,
    T = T,
    R = R,
    Q = Q,
    a1 = a1,
    P1 = P1,
)

In [ ]:
%%timeit

ret = CKF3.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF4.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF5.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF6.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF7.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF8.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF9.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF10.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF11.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF12.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF13.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF14.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF15.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF16.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF17.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF18.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF19.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF20.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF21.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF22.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF23.KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

In [ ]:
%%timeit

ret = CKF24.KalmanFilter(nny,Z,H,T,Q,a1,P1, R, True)

In [ ]:
%%timeit

ret = CKF25.KalmanFilter(nny,Z,H,T,Q,a1,P1, R, True)

## Tests #1 - Dot vs Dgemm

In [ ]:
%%cython 
import numpy as np
cimport numpy as np
import pandas as pd
cimport scipy.linalg.cython_blas as blas
import time

# def f(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
#       np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
#       np.ndarray[np.double_t,ndim=2, mode='fortran'] c,
#      ):

cdef f(double[::1,:] a, 
      double[::1,:] b,
       double[::1,:] c,
     ):

    cdef int m, n, k, lda, ldb, ldc
    cdef double alpha, beta

    alpha = 1.0
    beta = 0.0
    lda = 2
    ldb = 2
    ldc = 2
    m = 2
    n = 2
    k = 2

    blas.dgemm('n', #TRANSA
               'n', #TRANSB
               &m, #M
               &n, #N
               &k, #K
               &alpha, #ALPHA
               &a[0,0], #MAtrix A
               &lda, #LDA
               &b[0,0], #MAtrix B
               &ldb, #LDB
               &beta, #BETA
               &c[0,0], #Matrix C
               &ldc) #LDC

#     return c
    return c




#        blas.dgemm('n', #TRANSA
#               'n', #TRANSB
#               <int *> 2, #M
#               <int *> 2, #N
#               <int *> 2, #K
#               <double *> 1, #ALPHA
#               <double *> a_view, #MAtrix A
#               <int *> 2, #LDA
#               <double *> b_view, #MAtrix B
#               <int *> 2, #LDB
#               <double *> 0, #BETA
#               <double *> c_view, #Matrix C
#               <int *> 0) #LDC


cdef multDual(double[::1,:] a, 
      double[::1,:] b,):
    f(a,b,a)
    f(a,b,a)
    return a



def dotMult(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
      np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
      ):
    
#     n = 30
#     for i in range(0,1000):
#         for j in range (0,n):
    c = a.dot(b).dot(b)
    return c
            
def fMult(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
      np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
      ):
    
#     n = 30
#     for i in range(0,1000):
#         for j in range (0,n):
    f(a,b,a)
    f(a,b,a)
    return a
    
def l(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
      np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
      ):

    cdef np.ndarray[np.double_t,ndim=2, mode='fortran'] ac = a.copy(order='F')

#     c = np.empty((2, 2), float, order="F")
    
    timesDot = []
    timesC = []

    n = 30
    
    for i in range(0,10000):
        start_time = time.time()
        for j in range (0,n):
            c = a.dot(b).dot(b)
        end_time = time.time()

        timesDot.append(end_time - start_time)

        ac = a.copy(order='F')
        start_time = time.time()
        for j in range(0,n):
            f(ac,b,ac)
            f(ac,b,ac)
#         time.sleep(1)
        end_time = time.time()
    #     print("--- %s seconds ---" % (end_time - start_time))
        timesC.append(end_time - start_time)

    
#     return np.array(a)
    return {'dot':pd.DataFrame(timesDot), 
            'c' : pd.DataFrame(timesC),
            'final':np.array(ac)
           }



In [ ]:
a = np.array(np.random.rand(2,2),float,order='F')
b = np.array(np.random.rand(2,2),float, order='F')
c = np.empty((2,2),float,order='F')

In [ ]:
%%timeit

for i in range(0,100):

    a.dot(b).dot(b)

In [ ]:
%%timeit

for i in range(0,100):

    l(a.copy(order='F'), b.copy(order='F'))

In [ ]:
# a.dot(b).dot(a).dot(b).dot(b).dot(a).dot(a).dot(b).dot(b).dot(a)
a.dot(b).dot(b)

In [ ]:
ret = l(a.copy(order='F'), b.copy(order='F'))

In [ ]:
ret['final']

In [ ]:
dev = (ret['dot']-ret['c'])

In [ ]:
dev.describe()

# Test #2 - memory view vs ndarray

In [ ]:
%%cython

import numpy as np
cimport numpy as np

def main1(np.ndarray[np.double_t,ndim=2] a,np.ndarray[np.double_t,ndim=2] b):

    cdef np.ndarray[np.double_t,ndim=2] c = np.empty((2,2))

    for i in range(0,1000):
        c = a.dot(b)
    return c

    
def main2(np.ndarray[np.double_t,ndim=2] a,np.ndarray[np.double_t,ndim=2] b):

    cdef double[:,:] c = np.empty((2,2))

    for i in range(0,1000):
        c = a.dot(b)
    return c    
    
# def main2(double[:,:] a,double[:,:] b):

#     cdef double[:,:] c = np.empty((2,2))

#     for i in range(0,100):
#         c = a.dot(b)
#         return c
    

In [ ]:
a = np.array(np.random.rand(2,2),float)
b = np.array(np.random.rand(2,2),float)

In [ ]:
%%timeit 

main1(a,b)

In [ ]:
%%timeit 

main2(a,b)

Next Steps:

separate test 1 in two functions
disable gil, and decorators


# Test #3 - Decorators

In [ ]:
%%cython -a
# %%cython 

import numpy as np
cimport numpy as np
import pandas as pd
cimport scipy.linalg.cython_blas as blas
import time
cimport cython

# def f(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
#       np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
#       np.ndarray[np.double_t,ndim=2, mode='fortran'] c,
#      ):


@cython.boundscheck(False)  
@cython.wraparound(False)
@cython.initializedcheck(False)
@cython.cdivision(True)
@cython.nonecheck(False)
cdef void f(double[::1,:] a, 
      double[::1,:] b,
       double[::1,:] c,
     ) nogil:

    cdef int m, n, k, lda, ldb, ldc
    cdef double alpha, beta

    alpha = 1.0
    beta = 0.0
    lda = 2
    ldb = 2
    ldc = 2
    m = 2
    n = 2
    k = 2

    blas.dgemm('n', #TRANSA
               'n', #TRANSB
               &m, #M
               &n, #N
               &k, #K
               &alpha, #ALPHA
               &a[0,0], #MAtrix A
               &lda, #LDA
               &b[0,0], #MAtrix B
               &ldb, #LDB
               &beta, #BETA
               &c[0,0], #Matrix C
               &ldc) #LDC

#     return c
#     return c




#        blas.dgemm('n', #TRANSA
#               'n', #TRANSB
#               <int *> 2, #M
#               <int *> 2, #N
#               <int *> 2, #K
#               <double *> 1, #ALPHA
#               <double *> a_view, #MAtrix A
#               <int *> 2, #LDA
#               <double *> b_view, #MAtrix B
#               <int *> 2, #LDB
#               <double *> 0, #BETA
#               <double *> c_view, #Matrix C
#               <int *> 0) #LDC



@cython.boundscheck(False)  
@cython.wraparound(False)
@cython.initializedcheck(False)
@cython.cdivision(True)
@cython.nonecheck(False)
def dotMult(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
      np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
      ):
    
#     n = 30
    for i in range(0,10000):
#         for j in range (0,n):
        c = a.dot(b).dot(b)
    return c
         
@cython.boundscheck(False)  
@cython.wraparound(False)
@cython.initializedcheck(False)
def fMult(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
      np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
      ):
    
#     n = 30
    for i in range(0,10000):
#         for j in range (0,n):
        f(a,b,a)
        f(a,b,a)
    return a
    
def l(np.ndarray[np.double_t,ndim=2, mode='fortran'] a, 
      np.ndarray[np.double_t,ndim=2, mode='fortran'] b,
      ):

    cdef np.ndarray[np.double_t,ndim=2, mode='fortran'] ac = a.copy(order='F')

#     c = np.empty((2, 2), float, order="F")
    
    timesDot = []
    timesC = []

    n = 30
    
    for i in range(0,100000):
        start_time = time.time()
        for j in range (0,n):
            c = a.dot(b).dot(b)
        end_time = time.time()

        timesDot.append(end_time - start_time)

        ac = a.copy(order='F')
        start_time = time.time()
        for j in range(0,n):
            f(ac,b,ac)
            f(ac,b,ac)
#         time.sleep(1)
        end_time = time.time()
    #     print("--- %s seconds ---" % (end_time - start_time))
        timesC.append(end_time - start_time)

    
#     return np.array(a)
    return {'dot':pd.DataFrame(timesDot), 
            'c' : pd.DataFrame(timesC),
            'final':np.array(ac)
           }




In [ ]:
a = np.array(np.random.rand(2,2),float,order='F')
b = np.array(np.random.rand(2,2),float, order='F')
c = np.empty((2,2),float,order='F')

In [ ]:
a.dot(b).dot(b)

In [ ]:
dotMult(a,b)

In [ ]:
fMult(a,b)

In [ ]:
%%timeit

dotMult(a,b)

In [ ]:
%%timeit

fMult(a,b)

# Line Profiler

In [ ]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1

import numpy as np
cimport numpy as np
import pandas as pd
import time
from cpython cimport bool
import line_profiler
cimport cython

from libc.math cimport isnan
from libc.math cimport log
from libc.math cimport M_PI

ctypedef np.double_t DTYPE_t

##### KF
# Univariate version of durbin and koopman
# Should be the same as CKF24 with a constant defined



@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline int getSum(double [:,:] arr,int t, int p) nogil:

    cdef int i
    cdef int s = 0

    for i in range(0,p):
#        print(arr[i])
        if not isnan(arr[t,i]):
             s += 1
    return s


@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double yZa(double [:] arr1, double [:] arr2 , int m, double y) nogil:

    cdef double res = 0
    cdef int i

    for i in range(0,m):
        res += arr1[i] * arr2[i]
    return y-res


@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline void outerProductP(double [:] arr1, double [:] arr2 , int m, double [:,:] res, double F) nogil:

#    cdef double[:,:] res
    cdef int i
    cdef int j

    for i in range(0,m):
        for j in range(0,m):
            res[i,j] -= arr1[i] * arr1[j] * F
#    return res



@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void PZF(double[:] K, double F, double[:,:] P, double[:] Zti ,
int n, int p, int m) nogil:
#Note n, m and p are the generical matrix sizes from
#https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm

#K[:] = P[:,:].dot(Zt[i]) * (1/F)
#K(m x 1) = P(m x m) * Zt[i](m x 1) * (1/F) (1 x 1)
#Matrix = problem
# n = m
# p = 1
# m = m

    cdef double invF = 1/F
    for i in range(0,n):
        K[i] = 0
        for k in range(0,m):
            K[i] += P[i,k] * Zti[k]
        K[i] *= invF



@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef inline void AKv(double[:] a1, double[:] a0 , double[:] K, double v ,
int m) nogil:

#a[t,i+1,:] = a[t,i,:] + K[:] * v
#AKv(a_mv[t,i+1,:], a_mv[t,i,:], K_mv[:], v, m)

    for i in range(0,m):
        a1[i] = a0[i] + K[i] * v



@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline double ZPZH(double[:] Zt, double[:,:] P, double Ht, int m) nogil:

#F = Zt[i].dot(P[:,:]).dot(Zt[i]) + Ht[i]
#F(1 x 1) = Zt[i] (1 x m) * P (m x m) * Zt[i] (m x 1) + Ht[i] (1 x 1)

    cdef double F = 0
    cdef double temp = 0
    for i in range(0,m):
        temp = 0
        for j in range(0,m):
            temp += Zt[j]*P[j,i]

        F += temp * Zt[i]

    return F + Ht


@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline void TPT(double[:,:] T, double[:,:] P, double[:,:] TT,
double[:,:] RQR, int m, double[:,:] temp) nogil:

# P[:, :] = T.dot(P[:,:]).dot(TT) + RQR
# P(m x m) = T(m x m) P(m x m) TT(m x m) + RQR (m x m)
# Will do 2 matrix multiplications, can be done better, by doing it in a single
#operation


    cdef double tempk2, tempk1

    for i in range(0,m):
        for j in range(0,m):
            tempk1 = 0
            for k1 in range(0,m):
                tempk2 = 0
                for k2 in range(0,m):
                    tempk2 += T[i,k2] * P[k2,k1]
                tempk1 += tempk2 * TT[j,i]
            temp[i,j] = tempk1

    for i in range(0,m):
        for j in range(0,m):
            P[i,j] = temp[i,j] + RQR[i,j]



@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline void Ta(double[:] a1, double[:] a0 , double[:,:] T, int m) nogil:

# a[t+1,0,:] =  T.dot(a[t, pt, :])
#a(m x 1) = T(m x m) * a(m x 1)
    for i in range(0,m):
        a1[i] = 0
        for j in range(0,m):
            a1[i] += T[i,j] * a0[j]



@cython.boundscheck(False)
@cython.wraparound(False)
cdef inline int indexNans(double[:] yt, double[:] yt_orig,double[:,:] Zt, double[:,:] Zt_orig, double[:] H, double[:] H_orig, int p) nogil:
#yt[:pt] = y[t,yind]
#Zt[:pt,:] = Z[yind,:]
#Ht[:pt] = H[yind]
#should do all that and return pt
#yt(p x 1)
#Zt(p x p)

    cdef int i,j
    cdef int pt = 0

    for i in range(0,p):
        if not isnan(yt_orig[i]):
            yt[pt] = yt_orig[i]
            H[pt] = H_orig[i]

            for j in range(0,p):
                Zt[pt,j] = Zt_orig[i,j]

            pt += 1

    return pt




#@cython.initializedcheck(False)
#@cython.nonecheck(False)
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
def KalmanFilter(
        np.ndarray[DTYPE_t, ndim=2] y,
        np.ndarray[DTYPE_t, ndim=2] Z,
        np.ndarray[DTYPE_t, ndim=2] Hsq,
        np.ndarray[DTYPE_t, ndim=2] T,
        np.ndarray[DTYPE_t, ndim=2] Q,
        np.ndarray[DTYPE_t, ndim=1] a1,
        np.ndarray[DTYPE_t, ndim=2] P1,
        np.ndarray[DTYPE_t, ndim=2] R,
        likelihood = True,
   ) :

    # p = number of variables in Yt
    # y should be (n x p)

    cdef int n,p,m,t,i,pt

    n = y.shape[0]
    p = y.shape[1]
    m = a1.shape[0] #number of states


    cdef np.ndarray[DTYPE_t,ndim=3] a = np.empty((n+1,p+1,m))
    a[0,0,:] = a1

    cdef np.ndarray[DTYPE_t,ndim=2] P = np.empty((m, m))
    P[:,:] = P1

    cdef np.ndarray[DTYPE_t,ndim=2] mmTemp = np.empty((m, m))

    cdef np.ndarray[DTYPE_t,ndim=1] K = np.empty((m))

    cdef double v,F


    # RQR = np.linalg.multi_dot([R, Q, R.T])
    cdef np.ndarray[DTYPE_t,ndim=2] RQR = R.dot(Q).dot(R.T)

    cdef np.ndarray[DTYPE_t,ndim=2] TT = T.T

    yhat = np.empty((n,p)) #Later I should use it to export in numpy not pandas


    cdef np.ndarray[DTYPE_t,ndim=1] yt = np.empty(p)
    cdef np.ndarray[DTYPE_t,ndim=1] yInd = np.empty(p)
    cdef np.ndarray[DTYPE_t,ndim=2] Zt = np.empty((p, m))
    cdef np.ndarray[DTYPE_t,ndim=1] Ht = np.empty(p)


    cdef np.ndarray[DTYPE_t,ndim=1] H = np.diag(Hsq) #ONLY WORKS FOR DIAGONAL H

#    cdef np.ndarray[DTYPE_t,ndim=2] y_orig = y.copy()
    cdef np.ndarray[DTYPE_t,ndim=2] Z_orig = Z.copy()
    cdef np.ndarray[DTYPE_t,ndim=1] H_orig = H.copy()


    cdef double[:,:] y_mv = y
#    cdef double[:,:] y_orig_mv = y_orig
    cdef double[:,:] Z_orig_mv = Z_orig
    cdef double[:] H_orig_mv = H_orig
    cdef double[:] yt_mv = yt
    cdef double[:] yInd_mv = yInd
    cdef double[:,:,:] a_mv = a
    cdef double[:,:] Zt_mv = Zt
    cdef double[:] K_mv = K
    cdef double[:,:] P_mv = P
    cdef double[:] Ht_mv = Ht
    cdef double[:,:] T_mv = T
    cdef double[:,:] TT_mv = TT
    cdef double[:,:] RQR_mv = RQR
    cdef double[:,:] mmTemp_mv = mmTemp

    cdef double pst
    cdef double pstSum
    cdef double ll = 0
    cdef double lpi= log(2*M_PI)

    for t in range(0, n):


#        pt = getSum(y_mv, t, p)
#
#        yt[:pt] = y[t,yind]
#        Zt[:pt,:] = Z[yind,:]
#        Ht[:pt] = H[yind] #ONLY WORKS FOR DIAGONAL H

        pt = indexNans(yt_mv, y_mv[t,:],Zt_mv[:,:], Z_orig_mv[:,:],Ht_mv[:],H_orig_mv[:], p)

        pst = 0


        for i in range(0, pt):

#            v = yt[i] - np.dot(Zt[i], a[t,i,:])
#            v = yt_mv[i] - innerProduct(Zt_mv[i],a_mv[t,i,:],m)
            v =  yZa(Zt_mv[i],a_mv[t,i,:],m, yt_mv[i])

            # F[t,i] = np.linalg.multi_dot([Z[i], P[t, i,:,:], Z[i]]) + H[i, i]
#            F = Zt[i].dot(P[:,:]).dot(Zt[i]) + Ht[i]
            F = ZPZH(Zt_mv[i], P_mv[:,:], Ht_mv[i], m)

            if likelihood:
                if F>0:
                    ll += log(F) + (v ** 2) / F
                    pst += 1

#            K[:] = P[:,:].dot(Zt[i]) * (1/F)
            PZF(K_mv[:], F, P_mv[:,:], Zt_mv[i] , m, 1, m)

#            a[t,i+1,:] = a[t,i,:] + K[:] * v
            AKv(a_mv[t,i+1,:], a_mv[t,i,:], K_mv[:], v, m)

#            P[:,:] += - np.outer(K * F, K)
#            P[:,:] += - np.dot(K[:,None] * F, K[None,:])
            outerProductP(K_mv,K_mv,m,P_mv, F)

        if likelihood:
            ll += pst * lpi

#        a[t+1,0,:] =  T.dot(a[t, pt, :])
        Ta(a_mv[t+1,0,:], a_mv[t, pt, :], T_mv, m)

        # P[t+1, 0,:,:] = np.linalg.multi_dot([T, P[t, i + 1,:,:], TT]) + RQR
#        P[:, :] = T.dot(P[:,:]).dot(TT) + RQR
        TPT(T_mv, P_mv, TT_mv, RQR_mv, m, mmTemp_mv)

        # times.append(temp1 == temp2)


    alpha = a[:n, 0,:]
    yhat = pd.DataFrame(np.dot(Z, alpha.T).T)
    ll *= -0.5
    # pd.DataFrame(times).to_pickle("Mult")
    # pd.DataFrame(times).to_pickle("Dot")

    if likelihood:
        return ll

    return yhat



In [ ]:
profile = line_profiler.LineProfiler(KalmanFilter)
profile.runcall(KalmanFilter, nny,Z,H,T,Q,a1,P1, R)
profile.print_stats()

In [ ]:
ret = KalmanFilter(nny,Z,H,T,Q,a1,P1, R)

vy = pd.DataFrame(y)

i = 0

pd.concat([vy[i], ret[i]], axis=1).plot()


In [ ]:
%%cython 

import numpy as np
cimport numpy as np


# cdef void func(double* letsee):
#     for i in range(0,4):
# #         if not isnan(letsee[i]):
#             print(letsee[i])
#             print(isnan(letsee[i]))

def main1():
#     cdef np.ndarray[np.double_t,ndim=2] a = np.array([[1,np.nan,3,4],[1,np.nan,3,4]])

    n = 10
    cdef np.ndarray[np.double_t,ndim=2] a = np.random.randn(n,n)
    
    cdef np.ndarray[np.double_t,ndim=2] b
    
    cdef double[:,:] a_mv = a
    
    ind = np.ones(n)
    ind[3] = np.nan
    ind[5] = np.nan
    ind[7] = np.nan
    
    
    print(np.array(a_mv))
    print("Hey")
    print(np.array(a_mv[5,(1,3)]))
    
    
#     cdef int[:] intInd = np.array([0,1,2,4,6,8,9])
    
#     for i in range(0,100):
#         b = a[5,ind]
        
    
    
#     return b

# def main2():
# #     cdef np.ndarray[np.double_t,ndim=2] a = np.array([[1,np.nan,3,4],[1,np.nan,3,4]])

#     n = 10
#     cdef np.ndarray[np.double_t,ndim=2] a = np.random.randn(n,n)
    
#     cdef np.ndarray[np.double_t,ndim=2] b
    
#     cdef double[:,:] a_mv = a
    
#     ind = np.ones(n)
#     ind[3] = np.nan
#     ind[5] = np.nan
#     ind[7] = np.nan
    
#     cdef int[:] intInd = np.array([0,1,2,4,6,8,9])
    
#     for i in range(0,100):
#         b = a_mv[5,intInd]
        
    
    
#     return b

In [ ]:
main1()

In [ ]:
a = np.empty(2)
a[:] = np.array([1,2,3])

In [ ]:
x = np.random.randn(5,5)
x

In [ ]:
x[2,[1,2]]

In [ ]:
%%cython 
cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport isnan



def main(np.ndarray[double,ndim=1] a, int n):
    cdef double[:] a_mv = a
    for i in range(0,n):
        print(a_mv[i])
        if isnan(a_mv[i]):
            print("pa")

# @cython.cdivision(True)
# @cython.boundscheck(False)
# @cython.wraparound(False)
# def main2(np.ndarray[double,ndim=2] a,np.ndarray[double,ndim=2] b, int n):
    
#     cdef double[:,:] a_mv = a
#     cdef double[:,:] b_mv = b
#     cdef double res = 0
    
#     for i in range(0,n):
#         res += np.dot(a_mv[5,:],b_mv[5,:])

#     return res
        

In [ ]:
a = np.random.randn(10,)
b = np.random.randn(10,10)
n = 1000

In [ ]:
a = np.array([1,np.nan,3,4,5],dtype=np.float64)
n = 5
main(a,n)

In [ ]:
main1(a,b,n)

In [ ]:
main2(a,b,n)

In [ ]:
%timeit main1(a,b,n)
%timeit main2(a,b,n)

In [ ]:
x = np.array([1,2,3,9,20])
y = np.array([7,8,9,13,133])




In [ ]:
res = np.outer(x,y)
res

In [ ]:
i = 4
j = 2
res[i,j]

In [ ]:
x[i]*y[j]

In [ ]:
%%cython

from libc.math cimport M_PI

print(M_PI)